In [45]:
from collections import Counter
from cltk.stem.lemma import LemmaReplacer
from cltk.stem.latin.j_v import JVReplacer
from cltk.corpus.readers import get_corpus_reader

latinlibrary = get_corpus_reader(language='latin', corpus_name='latin_text_latin_library')

jv_replacer = JVReplacer()

skip_chars = '0123456789`~!@#$%^&*()))_+-={}[]|\:;"\'<>?,./'

def clean_text(text):
    table = {ord(char): None for char in skip_chars}
    return text.translate(table)

nonword_name_abbreviations = 'Ap,A,D,F,C,Cn,H,L,Mai,Mam,M,Min,N,Oct,P,Q,Seq,Ser,Sp,St,T,Ti,V,Vol,Vop'.lower().split(',')

def clean_entry(entry):
    if (entry in nonword_name_abbreviations):
        return entry.capitalize()+'.'
    return ''.join(filter(lambda x: x.isalpha(), entry))

def get_corpus_files(author):
    fileids = filter(lambda s: s.startswith(author), latinlibrary.fileids())
    files = [latinlibrary.abspath(fileid) for fileid in fileids]
    return files

def merge_file_contents(paths):
    text = ''
    for path in paths:
        f = open(path)
        text += f.read()
        #print('loaded', path, 'buffer length', len(text))
        f.close()
    return text


In [46]:
# We need to import a data model to train the lemmatizer.

import os
from cltk.utils.file_operations import open_pickle

# Set up training sentences

rel_path = os.path.join('~/cltk_data/latin/model/latin_models_cltk/lemmata/backoff')
path = os.path.expanduser(rel_path)

# Check for presence of latin_pos_lemmatized_sents
file = 'latin_pos_lemmatized_sents.pickle'      

latin_pos_lemmatized_sents_path = os.path.join(path, file)
if os.path.isfile(latin_pos_lemmatized_sents_path):
    latin_pos_lemmatized_sents = open_pickle(latin_pos_lemmatized_sents_path)
else:
    latin_pos_lemmatized_sents = []
    print('The file %s is not available in cltk_data' % file)  
    
# Set up CLTK Latin backoff lemmatizer
from cltk.lemmatize.latin.backoff import BackoffLatinLemmatizer
lemmatizer = BackoffLatinLemmatizer(latin_pos_lemmatized_sents)

In [54]:
from cltk.tokenize.word import WordTokenizer

authors = (
    'caesar',
    'livy',
    'ovid',
    'bacon',
    'jerome',
    'aquinas',
    'bacon',
    'newton'
)
word_tokenizer = WordTokenizer('latin')

data = {}
for author in authors:
    freq_table = []

    files = get_corpus_files(author)
    text = merge_file_contents(files)
    text = clean_text(text)

#     tokens = word_tokenizer.tokenize(jv_replacer.replace(text).lower())
    tokens = word_tokenizer.tokenize(text.lower())
    lemmataPairs = lemmatizer.lemmatize(tokens)
    lemmata = [pair[1] for pair in lemmataPairs]
    N = len(lemmata)
    longest_token = max(lemmata, key=len)
    freq_count = Counter(lemmata)
    data[author] = freq_count

In [ ]:
def percent(n, decimal=2):
    return round(n * 100, decimal)

def normalize(freq_table):
    result = freq_table.copy()
    N = sum(freq_table.values())
    for lemma in result:
        result[lemma] /= N
    return result

def normalize_all(data):
    result = data.copy()
    for author in result:
        result[author] = normalize(result[author])
    return result

def merge_all(data):
    result = Counter()
    for author in data:
        result += data[author]
    return result

def word_use_detail(word, author):
    author_data = data[author]
    N = sum(author_data.values())
    return author[0:3] + ':' + str(percent(author_data[word] / N, 2))

PERIOD = 1
LAST_WORD = 50000
SHOW_DETAILS = False

def print_counter_data(counter):
    f = open('+'.join(authors)+'-'+str(LAST_WORD)+'.txt', 'w')
    coverage = 0
    N = sum(counter.values())
    index = 1
    for word, freq in counter.most_common(LAST_WORD):
        coverage += freq / N
        if index == 1 or index % PERIOD == 0:

            details = ' '.join([word_use_detail(word, author) for author in authors])
            segs = (
                index,
                clean_entry(word),
                percent(freq, 6),
                percent(coverage, 2),
                details if SHOW_DETAILS else '',
            )
            line = ' '.join((
                '%5s' % segs[0],
                '%-15s' % segs[1],
                '%7s' % segs[2],
                '(%5s)' % segs[3],
                segs[4],
            ))
            print(line)
            f.write('\t'.join([str(seg) for seg in segs])+'\n')
        index += 1
    f.close()
normalized_combined_freq_table = normalize(merge_all(data))
print_counter_data(normalized_combined_freq_table)

    1 sum             3.809928 ( 3.81) 
    2 et              2.751495 ( 6.56) 
    3 qui             2.646158 ( 9.21) 
    4 in              2.35953 (11.57) 
    5 que             1.668215 (13.24) 
    6 is              1.345433 (14.58) 
    7 ad              1.250064 (15.83) 
    8 non             1.086925 (16.92) 
    9 cum             0.926613 (17.84) 
   10 ab              0.886888 (18.73) 
   11 ut              0.871935 ( 19.6) 
   12 hic             0.830053 (20.43) 
   13 sui             0.632769 (21.07) 
   14 sed             0.575116 (21.64) 
   15 suus            0.574372 (22.22) 
   16 omnis           0.568569 (22.78) 
   17 res             0.530258 (23.31) 
   18 ex              0.528547 (23.84) 
   19 neque           0.522745 (24.37) 
   20 dico            0.513967 (24.88) 
   21 per             0.50809 (25.39) 
   22 ne              0.501097 (25.89) 
   23 possum          0.474614 (26.36) 
   24 de              0.458843 (26.82) 
   25 atque           0.44999 (27.27) 
   

  350 impetus         0.039502 (61.23) 
  351 accido          0.039278 (61.27) 
  352 metus           0.038906 (61.31) 
  353 porta           0.038386 (61.35) 
  354 humanus         0.038311 (61.38) 
  355 sacer           0.038311 (61.42) 
  356 inquantum       0.038163 (61.46) 
  357 pertineo        0.038088 ( 61.5) 
  358 proficiscor     0.038014 (61.54) 
  359 addo            0.038014 (61.58) 
  360 dimitto         0.037939 (61.61) 
  361 flumen          0.037865 (61.65) 
  362 quattuor        0.037865 (61.69) 
  363 mos             0.037865 (61.73) 
  364 mons            0.037716 (61.76) 
  365 scientia        0.037642 ( 61.8) 
  366 regius          0.037642 (61.84) 
  367 tollo           0.037493 (61.88) 
  368 aer             0.037493 (61.91) 
  369 tamquam         0.037344 (61.95) 
  370 articulus       0.037344 (61.99) 
  371 sumo            0.03727 (62.03) 
  372 reddo           0.03727 (62.06) 
  373 ostendo         0.03727 ( 62.1) 
  374 uir             0.037195 (62.14) 
  3

  687 quivis          0.020234 (70.46) 
  688 sanctus         0.020234 (70.48) 
  689 genero          0.020234 ( 70.5) 
  690 fundo           0.02016 (70.52) 
  691 memoria         0.02016 (70.54) 
  692 claudo          0.02016 (70.56) 
  693 operatio        0.02016 (70.58) 
  694 remitto         0.020086 ( 70.6) 
  695 omnino          0.020011 (70.62) 
  696 permitto        0.020011 (70.64) 
  697 tendo           0.020011 (70.66) 
  698 contendo        0.019937 (70.68) 
  699 parum           0.019862 ( 70.7) 
  700 repeto          0.019862 (70.72) 
  701 factum          0.019788 (70.74) 
  702 occurro         0.019788 (70.76) 
  703 salus           0.019788 (70.78) 
  704 aeger           0.019714 ( 70.8) 
  705 error           0.019714 (70.82) 
  706 dolor           0.019639 (70.84) 
  707 pecco           0.019416 (70.86) 
  708 forum           0.019416 (70.88) 
  709 consequor       0.019342 ( 70.9) 
  710 morior          0.019342 (70.92) 
  711 numen           0.019342 (70.94) 
  71

 1004 graecia         0.013465 (75.62) 
 1005 praetereo       0.013465 (75.63) 
 1006 requiro         0.013465 (75.65) 
 1007 illic           0.013465 (75.66) 
 1008 uoco            0.013465 (75.67) 
 1009 dimico          0.01339 (75.69) 
 1010 cedo            0.01339 ( 75.7) 
 1011 huc             0.01339 (75.72) 
 1012 floreo          0.01339 (75.73) 
 1013 ordino          0.01339 (75.74) 
 1014 peruenio        0.01339 (75.76) 
 1015 frustra         0.013316 (75.77) 
 1016 promitto        0.013316 (75.78) 
 1017 mensis          0.013316 ( 75.8) 
 1018 prout           0.013316 (75.81) 
 1019 livy            0.013316 (75.82) 
 1020 tellus          0.013316 (75.84) 
 1021 emitto          0.013242 (75.85) 
 1022 paene           0.013242 (75.86) 
 1023 consulatus      0.013242 (75.87) 
 1024 livi            0.013242 (75.89) 
 1025 noto            0.013167 ( 75.9) 
 1026 maritimus       0.013167 (75.91) 
 1027 dubito          0.013167 (75.93) 
 1028 ripa            0.013167 (75.94) 
 1029 

 1324 daemon          0.009968 (79.29) 
 1325 incolumis       0.009894 ( 79.3) 
 1326 aequo           0.009894 (79.31) 
 1327 consensus       0.009894 (79.32) 
 1328 edico           0.009894 (79.33) 
 1329 recedo          0.009894 (79.34) 
 1330 sulpicius       0.009894 (79.35) 
 1331 canis           0.009894 (79.36) 
 1332 pavor           0.009894 (79.37) 
 1333 notio           0.009894 (79.38) 
 1334 deicio          0.00982 (79.39) 
 1335 indicium        0.00982 ( 79.4) 
 1336 converro        0.00982 (79.41) 
 1337 intueor         0.00982 (79.42) 
 1338 impleo          0.00982 (79.43) 
 1339 semen           0.00982 (79.44) 
 1340 quantitas       0.00982 (79.45) 
 1341 cum             0.009745 (79.46) 
 1342 judico          0.009745 (79.47) 
 1343 moles           0.009745 (79.48) 
 1344 cingo           0.009745 (79.49) 
 1345 decipio         0.009745 ( 79.5) 
 1346 quinquaginta    0.009745 (79.51) 
 1347 elephantus      0.009745 (79.52) 
 1348 mortuus         0.009745 (79.53) 
 1349 d

...

12410 subsistentiae   0.000298 (97.42) 
12411 motio           0.000298 (97.42) 
12412 assistentes     0.000298 (97.42) 
12413 relativam       0.000298 (97.42) 
12414 essentialium    0.000298 (97.42) 
12415 comprehensor    0.000298 (97.42) 
12416 arii            0.000298 (97.42) 
12417 hilarium        0.000298 (97.42) 
12418 platonicorum    0.000298 (97.42) 
12419 distinguerentur 0.000298 (97.43) 
12420 imagino         0.000298 (97.43) 
12421 instrumental    0.000298 (97.43) 
12422 coaeterna       0.000298 (97.43) 
12423 adiectivum      0.000298 (97.43) 
12424 doctr           0.000298 (97.43) 
12425 attributor      0.000298 (97.43) 
12426 attribuendi     0.000298 (97.43) 
12427 consubstantialitas 0.000298 (97.43) 
12428 aeternaliter    0.000298 (97.43) 
12429 universaliores  0.000298 (97.43) 
12430 incoeperunt     0.000298 (97.43) 
12431 incorruptibilitas 0.000298 (97.43) 
12432 irrationalia    0.000298 (97.43) 
12433 comestio        0.000298 (97.43) 
12434 commensuretur   0.000298 (97.

12720 uxellodunum     0.000223 ( 97.5) 
12721 derivaris       0.000223 ( 97.5) 
12722 prorumpo        0.000223 ( 97.5) 
12723 triclinium      0.000223 ( 97.5) 
12724 fundanius       0.000223 ( 97.5) 
12725 arcuballista    0.000223 ( 97.5) 
12726 necopinantibus  0.000223 ( 97.5) 
12727 ursaonem        0.000223 ( 97.5) 
12728 patrocinaris    0.000223 ( 97.5) 
12729 resina          0.000223 ( 97.5) 
12730 nardus          0.000223 ( 97.5) 
12731 antenorem       0.000223 ( 97.5) 
12732 rutulorum       0.000223 ( 97.5) 
12733 capys           0.000223 ( 97.5) 
12734 summersus       0.000223 ( 97.5) 
12735 proco           0.000223 ( 97.5) 
12736 stagnum         0.000223 ( 97.5) 
12737 relictaris      0.000223 ( 97.5) 
12738 crustumini      0.000223 ( 97.5) 
12739 antemnates      0.000223 ( 97.5) 
12740 vnam            0.000223 ( 97.5) 
12741 raptaris        0.000223 ( 97.5) 
12742 volgaris        0.000223 ( 97.5) 
12743 vtro            0.000223 ( 97.5) 
12744 tarpeius        0.000223 ( 97.5) 


13025 jugarius        0.000223 (97.56) 
13026 sacrificulis    0.000223 (97.56) 
13027 vaticinus       0.000223 (97.56) 
13028 papiri          0.000223 (97.56) 
13029 marcelle        0.000223 (97.56) 
13030 pluuit          0.000223 (97.56) 
13031 uolito          0.000223 (97.56) 
13032 aedituus        0.000223 (97.56) 
13033 comprensos      0.000223 (97.57) 
13034 stips           0.000223 (97.57) 
13035 paeligna        0.000223 (97.57) 
13036 uibius          0.000223 (97.57) 
13037 thurinis        0.000223 (97.57) 
13038 iocur           0.000223 (97.57) 
13039 discidium       0.000223 (97.57) 
13040 convello        0.000223 (97.57) 
13041 herdonea        0.000223 (97.57) 
13042 praecipitaris   0.000223 (97.57) 
13043 nasso           0.000223 (97.57) 
13044 inlacrimo       0.000223 (97.57) 
13045 diluceo         0.000223 (97.57) 
13046 serva           0.000223 (97.57) 
13047 fonteio         0.000223 (97.57) 
13048 uesper          0.000223 (97.57) 
13049 circumicio      0.000223 (97.57) 


13371 pyrrheum        0.000223 (97.64) 
13372 asser           0.000223 (97.64) 
13373 maritumae       0.000223 (97.64) 
13374 pepigerat       0.000223 (97.64) 
13375 celaenis        0.000223 (97.64) 
13376 maeandri        0.000223 (97.64) 
13377 illiberal       0.000223 (97.64) 
13378 pamphyliam      0.000223 (97.64) 
13379 tolostobogiorum 0.000223 (97.64) 
13380 brenno          0.000223 (97.64) 
13381 byzantium       0.000223 (97.64) 
13382 nicomede        0.000223 (97.64) 
13383 aegyptios       0.000223 (97.64) 
13384 anticus         0.000223 (97.64) 
13385 eposognatum     0.000223 (97.64) 
13386 tectosagos      0.000223 (97.64) 
13387 thraeces        0.000223 (97.64) 
13388 turmaris        0.000223 (97.64) 
13389 ariarathis      0.000223 (97.64) 
13390 samaei          0.000223 (97.64) 
13391 dissido         0.000223 (97.65) 
13392 plebiscito      0.000223 (97.65) 
13393 milesium        0.000223 (97.65) 
13394 thraecum        0.000223 (97.65) 
13395 thraecas        0.000223 (97.65) 


13714 averno          0.000223 (97.72) 
13715 aegyptia        0.000223 (97.72) 
13716 turgesco        0.000223 (97.72) 
13717 amantum         0.000223 (97.72) 
13718 testis          0.000223 (97.72) 
13719 obduro          0.000223 (97.72) 
13720 inguinibus      0.000223 (97.72) 
13721 thebana         0.000223 (97.72) 
13722 aurigo          0.000223 (97.72) 
13723 hara            0.000223 (97.72) 
13724 graio           0.000223 (97.72) 
13725 sponda          0.000223 (97.72) 
13726 sulmonis        0.000223 (97.72) 
13727 lyaeus          0.000223 (97.72) 
13728 automedo        0.000223 (97.72) 
13729 tiphys          0.000223 (97.72) 
13730 ascreo          0.000223 (97.72) 
13731 instita         0.000223 (97.72) 
13732 phrygio         0.000223 (97.72) 
13733 spatiaris       0.000223 (97.72) 
13734 herculei        0.000223 (97.72) 
13735 rubra           0.000223 (97.72) 
13736 eoas            0.000223 (97.72) 
13737 euphrates       0.000223 (97.72) 
13738 aonio           0.000223 (97.72) 
